In [1]:
import json
jsonl_path = "h:/ML_Models/_gemma/customdocs/converted/dogs_dataset_english.jsonl"

def load_jsonl(path):
    try:
        with open(jsonl_path, 'r', encoding='utf-8') as file:
            print("Loading JSONL file...")
            data = [json.loads(line) for line in file]
        return data
    except FileNotFoundError:
        print(f"File not found: {jsonl_path}")
        data = []
    except Exception as e:
        print(f"Error loading JSONL file: {e}")
        data = []

def extract_documents_and_ids(data):
    breed_documents = []
    breed_ids = []
    metadatas = []
    for record in data: 
        input_text = record.get("input", "")
        breed_documents.append(input_text)
        index = input_text.find(" Nationality: ") # find first space
        if index != -1:
            id = input_text[:index] # take first word as id
        else:
            id = input_text
        breed_ids.append(id)

        index_nationality = input_text.find(". Origin") # find first space
        nationality = ""
        if index_nationality != -1:
            nationality = input_text[ index + len(" Nationality: ") : index_nationality].strip() # take text between "Nacionalidad" and ". Origen"
            metadatas.append({"source": "dogs_dataset.jsonl", "Nationality": nationality})
        else:
            metadatas.append({"source": "dogs_dataset.jsonl", "Nationality": "unknown: " + input_text[:index]})
    return breed_documents, breed_ids, metadatas


In [2]:
from langchain_core.documents import Document

data = load_jsonl(jsonl_path)
texts, ids, metadatas = extract_documents_and_ids(data)
print(f"Loaded {len(texts)} documents.")

documents = [Document(page_content=texts[i], metadata=metadatas[i]) for i in range(len(texts))]
print("documents created.")

Loading JSONL file...
Loaded 57 documents.
documents created.


In [3]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_milvus import Milvus

# Embedding model
embedding_model = HuggingFaceEmbeddings(model_name="multi-qa-mpnet-base-dot-v1")

# Connect to your existing collection
vector_store = Milvus(
    embedding_model,
    collection_name="Dogs_Breeds_milvus_EN_1",
    connection_args={"host": "localhost", "port": "19530"}
)

vector_store.add_documents(documents=documents, ids=ids)
print("Vector store loaded.")

C:\Users\admin\AppData\Local\Temp\ipykernel_14556\88413281.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="multi-qa-mpnet-base-dot-v1")
c:\personal\_python\fastapi\PetshopAPIPython\langchain_test\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Vector store loaded.


In [ ]:
results = vector_store.similarity_search(
    "rescue dogs",
    k=5,
    #expr='source == "tweet"', # search metadata
    # param=...  # Search params for the index type. TODO???
)
print(f"Search results: {len(results)}")
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

Search results: 5
* SAINT BERNARD Nationality: Switzerland. Origin: Very ancient origins, descending from the Tibetan Mastiff and the mastiffs brought by Romans to the Alps 2000 years ago. The modern Saint Bernard emerged around the year 1000 at the monastery on the Great Saint Bernard Pass, founded by Father Bernard of Menthon. Monks crossed the ancient mastiff with Great Danes and Pyrenean Mountain Dogs. Their careful selection led to this remarkable breed, first appearing on a Swiss coat of arms in 1350. Its fame as a rescue dog began in the mid-18th century. Description: A giant among dog breeds—robust, muscular, with a powerful head. Minimum height: 70 cm for males, 65 cm for females. Weight: 55–80 kg. Broad, slightly convex skull with visible wrinkles; straight nasal bridge and pronounced stop; muzzle wider than long; drooping upper lip; strong, complete teeth; medium-sized, slightly sunken eyes, positioned forward, dark brown. Medium ears hang to the sides; strong neck with dewl